# Importation des bibliothèques

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Départements et filières

In [ ]:
def get_department_links(BASE_URL="http://ensak.usms.ac.ma"):
    url = f"{BASE_URL}/ensak/departements/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    departments = []

    divs = soup.select("div.col-sm-6")
    for div in divs:
        a_tag = div.find("a")
        if a_tag:
            name = a_tag.find("button").text.strip() if a_tag.find("button") else "Nom non trouvé"
            link = a_tag["href"]
            if not link.startswith("http"):
                link = BASE_URL + link
            departments.append({"Nom": name, "Lien": link})
    return departments


def extract_department_info(dept):
    url = dept["Lien"]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    chef = "Non trouvé"
    adjoint = "Non trouvé"
    filieres = []
    professeurs = []

    tables = soup.find_all("table")
    if tables:
        # Table 1: Chef et adjoint
        rows = tables[0].find_all("tr")
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 2:
                key = cols[0].get_text(strip=True)
                value = cols[1].get_text(strip=True)
                if "Chef de département" in key:
                    chef = value if value != "-" else "N'existe pas"
                elif "Adjoint" in key:
                    adjoint = value if value != "-" else "N'existe pas"

        # Table 2: Professeurs
        if len(tables) > 1:
            rows = tables[1].find_all("tr")[1:]  # Ignorer l'en-tête
            for row in rows:
                cols = row.find_all("td")
                if len(cols) >= 2:
                    nom = cols[0].get_text(strip=True)
                    email_tag = cols[1].find("a")
                    email = email_tag.get_text(strip=True) if email_tag else "Email non trouvé"
                    professeurs.append({"Nom": nom, "Email": email})

    # Extraction des filières
    h4_tags = soup.find_all("h4")
    for h4 in h4_tags:
        filiere = h4.get_text(strip=True)
        if filiere:
            filieres.append(filiere)

    return {
        "Nom Département": dept["Nom"],
        "Lien": dept["Lien"],
        "Chef": chef,
        "Adjoint": adjoint,
        "Filières": filieres if filieres else ["Aucune filière"],
        "Professeurs": professeurs
    }


def main(fichier_json="departements_ENSAKH.json"):
    departments = get_department_links()
    all_data = []

    for dept in departments:
        print(f"🔎 Traitement du département : {dept['Nom']}")
        info = extract_department_info(dept)
        all_data.append(info)
        time.sleep(1)

    with open(fichier_json, "w", encoding="utf-8") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)

    print(f"\n✅ Données exportées dans le fichier JSON : '{fichier_json}'")


if __name__ == "__main__":
    main()


🔎 Traitement du département : Mathématiques et Informatique
🔎 Traitement du département : Génie Electrique
🔎 Traitement du département : Cybersécurité Réseaux et Télécommunications
🔎 Traitement du département : Génie des Procédés

✅ Données exportées dans le fichier JSON : 'departements_ENSAKH.json'


# Ensa en chiffres

In [4]:

def extract_all_counters(url="http://ensak.usms.ac.ma/ensak/", output_file='chiffres_ENSAKH.json'):
    # Obtenir le contenu HTML depuis l'URL
    response = requests.get(url)
    response.encoding = 'utf-8'  # Important pour bien gérer les accents

    if response.status_code != 200:
        print(f"Erreur lors de la requête HTTP : {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    data = []

    counter_blocks = soup.find_all('div', class_='wpsm_counterbox')

    for block in counter_blocks:
        number_tag = block.find('span', class_='counter')
        title_tag = block.find('h3', class_='wpsm_count-title')

        number = number_tag.text.strip() if number_tag else None
        title = title_tag.text.strip() if title_tag else None

        data.append({'Nombre': number, 'Titre': title})

    # Enregistrer dans un fichier JSON
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# Exemple d'exécution
extract_all_counters()


# Clubs 

In [5]:
def extraire_clubs_ensak(url="http://ensak.usms.ac.ma/ensak/"):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Erreur lors de la requête HTTP : {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    clubs_data = []

    # Sélectionner tous les blocs de clubs
    club_blocks = soup.select("div.col-xs-12.zoom.campus")

    for block in club_blocks:
        # Extraire le lien vers la page du club
        link_tag = block.find("a", class_="img-thumb")
        club_link = link_tag["href"] if link_tag and "href" in link_tag.attrs else None

        # Extraire le nom du club
        title_tag = block.find("h3")
        club_name = title_tag.get_text(strip=True) if title_tag else "Nom non trouvé"

        # Initialiser le résumé
        resume = "Résumé non trouvé"

        # Accéder à la page du club pour extraire le résumé
        if club_link:
            try:
                detail_response = requests.get(club_link)
                detail_response.raise_for_status()
                detail_soup = BeautifulSoup(detail_response.content, "html.parser")

                # Extraire le contenu de la section 'courses-info'
                content_divs = detail_soup.find_all("div", class_="courses-info")
                if len(content_divs) >= 3:
                    content_div = content_divs[2]  # la troisième div
                else:
                    content_div = None
                if content_div:
                    # Supprimer les images et les tableaux
                    for tag in content_div.find_all(["img", "table"]):
                        tag.decompose()
                    # Extraire le texte
                    resume = content_div.get_text(separator="\n", strip=True)
            except requests.RequestException as e:
                resume = f"Erreur lors de l'accès au résumé : {e}"

            # Pause pour éviter de surcharger le serveur
            time.sleep(1)

        clubs_data.append({
            "Nom du Club": club_name,
            "Lien": club_link,
            "Résumé": resume
        })

    # Enregistrer dans un fichier JSON
    with open("clubs_ENSAKH.json", "w", encoding="utf-8") as f:
        json.dump(clubs_data, f, ensure_ascii=False, indent=4)

    print("✅ Les informations des clubs ont été enregistrées dans 'clubs_ENSAKH.json'.")

# Exécuter la fonction
extraire_clubs_ensak()


✅ Les informations des clubs ont été enregistrées dans 'clubs_ENSAKH.json'.


# les professeurs

In [6]:

def extraire_enseignants(url="http://ensak.usms.ac.ma/ensak/our-instructors/", fichier_json="enseignants_ENSAKH.json"):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        prof_blocks = soup.select("div.col-xs-12.col-sm-4.teachers")
        profs_data = []

        for prof in prof_blocks:
            # Nom et lien
            name_tag = prof.select_one("h4.author-name a")
            nom = name_tag.text.strip() if name_tag else "Nom non trouvé"
            lien = name_tag["href"] if name_tag else "#"

            # Fonction / titre
            p_tags = prof.select("p")
            fonction = p_tags[0].text.strip() if len(p_tags) > 0 else "Fonction non trouvée"

            # Email
            email_tag = prof.select_one("p.email a")
            email = email_tag.text.strip() if email_tag else "Email non trouvé"

            # Description détaillée depuis la page du prof
            description = "Description non trouvée"
            if lien != "#":
                try:
                    detail_response = requests.get(lien)
                    detail_soup = BeautifulSoup(detail_response.content, "html.parser")
                    divs = detail_soup.select("section.co-author > div")
                    if len(divs) >= 3:
                        third_div = divs[2]  # le troisième div
                        p_tags_detail = third_div.find_all("p")
                        if len(p_tags_detail) >= 2:
                            description = p_tags_detail[1].get_text(strip=True)
                        else:
                            description = "Deuxième paragraphe non trouvé dans le 3e div"
                    else:
                        description = "Troisième div non trouvé"
                except Exception as e:
                    description = f"Erreur lors de l'accès à la page prof : {e}"

                time.sleep(1)  # Pause pour éviter surcharge

            profs_data.append({
                "Nom": nom,
                "Lien": lien,
                "Fonction": fonction,
                "Email": email,
                "Description": description
            })

        # Sauvegarde en JSON
        with open(fichier_json, "w", encoding="utf-8") as f:
            json.dump(profs_data, f, ensure_ascii=False, indent=4)

        print(f"✅ Les informations des enseignants ont été enregistrées dans '{fichier_json}'")

    except Exception as e:
        print(f"❌ Erreur lors de l'extraction : {e}")

# Lancer la fonction
extraire_enseignants()


✅ Les informations des enseignants ont été enregistrées dans 'enseignants_ENSAKH.json'


# formations: double diplomation 

In [11]:
def scrape_modalites_ecoles_grouped_list(url="http://ensak.usms.ac.ma/ensak/double-diplomation/", json_filename="ecoles_modalites_DD_ENSAKH.json"):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    paragraph = soup.find("p", style="margin-left: 30px;")
    if not paragraph:
        print("❌ Paragraphe non trouvé.")
        return []

    lines = [str(item).strip() for item in paragraph.decode_contents().split("<br/>") if item.strip()]
    data = {}

    for line in lines:
        soup_line = BeautifulSoup(line, "html.parser")
        i_tag = soup_line.find("i", class_="fa fa-arrow-right")

        if not i_tag:
            continue

        school_name = i_tag.previous_sibling.strip().lstrip("-").strip()
        modalities_text = i_tag.next_sibling.strip().rstrip(".")
        modalities_clean = [m.strip() for m in modalities_text.replace("et", ",").split(",") if m.strip()]

        if school_name not in data:
            data[school_name] = []
        data[school_name].extend(modalities_clean)

    # Ajout spécial POLYTECH Angers
    if "POLYTECH Angers" not in data:
        data["POLYTECH Angers"] = []
    data["POLYTECH Angers"].extend(["Double diplomation", "Master"])

    # Supprimer doublons et trier
    for school in data:
        data[school] = sorted(list(set(data[school])))

    # Transformer en liste de dicts avec clé "École" et "Modalités"
    data_list = [{"École": school, "Modalités": modalities} for school, modalities in sorted(data.items())]

    # Sauvegarde JSON
    with open(json_filename, "w", encoding="utf-8") as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

    print(f"✅ Données regroupées exportées dans '{json_filename}' au format liste d'objets.")

    

# Test
scrape_modalites_ecoles_grouped_list()


✅ Données regroupées exportées dans 'ecoles_modalites_DD_ENSAKH.json' au format liste d'objets.
